import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [10]:
#!pip install pyspark==3.1.3

  Created wheel for pyspark: filename=pyspark-3.1.3-py2.py3-none-any.whl size=214463488 sha256=1473d28c779461eb1b76eb5c09a373a0bab014257716ecf4d089ca33d265a969
  Stored in directory: c:\users\qwerty\appdata\local\pip\cache\wheels\d1\b2\3a\d019c9382e6429910cef8ef0aae6a1970bfbbba13562590d70
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5


In [2]:
!pip install findspark

  Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)


In [3]:
import findspark
findspark.init()

In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

create local SparkSession

In [5]:
spark = SparkSession.builder.appName("pyspark_basics").getOrCreate()

read csv with inferschema

In [6]:
df = spark.read.option("inferSchema",True) \
                .option("delimiter",",") \
                .csv("ds_salaries.csv")

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [7]:
df = spark.read.option("inferSchema",True) \
                .option("delimiter",",") \
                .csv("ds_salaries.csv")

write schema of scv on screen

In [8]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)



create schema of this scv

In [9]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("work_year", IntegerType(), True),
    StructField("experience_level", StringType(), True),
    StructField("employment_type", StringType(), True),
    StructField("job_title", StringType(), True),
    StructField("salary", IntegerType(), True),
    StructField("salary_currency", StringType(), True),
    StructField("salary_in_usd", IntegerType(), True),
    StructField("employee_residence", StringType(), True),
    StructField("remote_ratio", IntegerType(), True),
    StructField("company_location", StringType(), True),
    StructField("company_size", StringType(), True)
])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [10]:
df = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("ds_salaries.csv")

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [11]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [12]:
df.show(5)

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist| 70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer| 85000|            GBP|       109024|                GB|          50|              GB|

print data in dataframe using display(df.toPandas())

In [13]:
display(df.toPandas())

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [14]:
df_job_title = df.select("job_title").distinct()

print all rows from df_job_titles without truncating jobs

In [16]:
df_job_title.show(df_job_title.count(), truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
|Research S

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [17]:
df_analytic = df.groupBy("job_title").agg(
                                            max("salary_in_usd").alias("max_salary"),
                                            avg("salary_in_usd").alias("avg_salary"),
                                            min("salary_in_usd").alias("min_salary")
                                        ).orderBy("job_title")

print all rows from df_analytic without trancating jobs

In [18]:
df_analytic.show()

+--------------------+----------+------------------+----------+
|           job_title|max_salary|        avg_salary|min_salary|
+--------------------+----------+------------------+----------+
|3D Computer Visio...|      5409|            5409.0|      5409|
|        AI Scientist|    200000| 66135.57142857143|     12000|
|  Analytics Engineer|    205300|          175000.0|    135000|
|Applied Data Scie...|    380000|          175655.0|     54238|
|Applied Machine L...|    423000|         142068.75|     31875|
|     BI Data Analyst|    150000| 74755.16666666667|      9272|
|  Big Data Architect|     99703|           99703.0|     99703|
|   Big Data Engineer|    114047|           51974.0|      5882|
|Business Data Ana...|    135000|           76691.2|     18442|
| Cloud Data Engineer|    160000|          124647.0|     89294|
|Computer Vision E...|    125000|44419.333333333336|     10000|
|Computer Vision S...|    150000|105248.66666666667|     70000|
|        Data Analyst|    200000| 92893.

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [19]:
from pyspark.sql.functions import desc

df_analytic = df_analytic.withColumn(
    "row_id",
    row_number().over(Window.orderBy(desc("avg_salary")))
)

print all data from df_analytic

In [20]:
df_analytic.show()

+--------------------+----------+------------------+----------+------+
|           job_title|max_salary|        avg_salary|min_salary|row_id|
+--------------------+----------+------------------+----------+------+
| Data Analytics Lead|    405000|          405000.0|    405000|     1|
|Principal Data En...|    600000| 328333.3333333333|    185000|     2|
|Financial Data An...|    450000|          275000.0|    100000|     3|
|Principal Data Sc...|    416000|215242.42857142858|    148261|     4|
|Director of Data ...|    325000|          195074.0|    130026|     5|
|      Data Architect|    266400| 177873.9090909091|     90700|     6|
|Applied Data Scie...|    380000|          175655.0|     54238|     7|
|  Analytics Engineer|    205300|          175000.0|    135000|     8|
|     Data Specialist|    165000|          165000.0|    165000|     9|
|        Head of Data|    235000|          160162.6|     32974|    10|
|Machine Learning ...|    260000|          158412.5|     12000|    11|
|Data 

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [21]:
df_analytic = df_analytic.select("row_id", "job_title", "avg_salary", "min_salary", "max_salary")

print df_analytic now

In [22]:
df_analytic.show()

+------+--------------------+------------------+----------+----------+
|row_id|           job_title|        avg_salary|min_salary|max_salary|
+------+--------------------+------------------+----------+----------+
|     1| Data Analytics Lead|          405000.0|    405000|    405000|
|     2|Principal Data En...| 328333.3333333333|    185000|    600000|
|     3|Financial Data An...|          275000.0|    100000|    450000|
|     4|Principal Data Sc...|215242.42857142858|    148261|    416000|
|     5|Director of Data ...|          195074.0|    130026|    325000|
|     6|      Data Architect| 177873.9090909091|     90700|    266400|
|     7|Applied Data Scie...|          175655.0|     54238|    380000|
|     8|  Analytics Engineer|          175000.0|    135000|    205300|
|     9|     Data Specialist|          165000.0|    165000|    165000|
|    10|        Head of Data|          160162.6|     32974|    235000|
|    11|Machine Learning ...|          158412.5|     12000|    260000|
|    1

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [30]:
# define the window                    
window = Window.partitionBy('experience_level').orderBy(col('salary_in_usd').desc())

# create the df_exp_lvl dataframe
df_exp_lvl = df.withColumn('rank', row_number().over(window)) \
            .filter(col('rank') == 1) \
            .withColumnRenamed('salary', 'biggest_salary')


print here df_exp_lvl

In [53]:
display(df_exp_lvl.toPandas())

,id,work_year,experience_level,employment_type,job_title,biggest_salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,rank
0,252,2021,EX,FT,Principal Data Engineer,600000,USD,600000,US,100,US,L,1
1,33,2020,MI,FT,Research Scientist,450000,USD,450000,US,0,US,M,1
2,37,2020,EN,FT,Machine Learning Engineer,250000,USD,250000,US,50,US,L,1
3,63,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L,1


create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [66]:
df_max_salary_exp_lvl = df.groupBy("experience_level").agg(max("salary").alias("max_salary"))

df_best = df.join(
                df_max_salary_exp_lvl, 
                (df["salary"] == df_max_salary_exp_lvl["max_salary"])
                & (df["experience_level"] == df_max_salary_exp_lvl["experience_level"]),
                how="left_semi") \
            .select("id", "experience_level", col("salary").alias("biggest_salary"), "employee_residence")


print df_best

In [67]:
display(df_best.toPandas())

,id,experience_level,biggest_salary,employee_residence
0,16,EN,4450000,JP
1,177,MI,30400000,CL
2,285,SE,7000000,IN
3,384,EX,6000000,IN


drop duplicates if exist by experience_level

In [68]:
df_best = df_best.dropDuplicates(["experience_level"])

print df_best

In [69]:
display(df_best.toPandas())

,id,experience_level,biggest_salary,employee_residence
0,384,EX,6000000,IN
1,177,MI,30400000,CL
2,16,EN,4450000,JP
3,285,SE,7000000,IN


create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [70]:
df_new_best = df_best.withColumn('experience_level', 
                        when(col('experience_level') == 'MI', 'middle') \
                       .when(col('experience_level') == 'SE', 'senior') \
                       .otherwise(None)) \
                     .drop('id')

print df_new_best

In [71]:
display(df_new_best.toPandas())

,experience_level,biggest_salary,employee_residence
0,None,6000000,IN
1,middle,30400000,CL
2,None,4450000,JP
3,senior,7000000,IN


write df_new_best like 1.csv and load then it to df_final

In [72]:
df_new_best.write.format("csv").mode("overwrite").option("header", "true").save("1.csv")

In [92]:
df_final = spark.read.option("delimiter",",").option("header", "true").csv("1.csv")

print df_final

In [93]:
df_final.show()

+----------------+--------------+------------------+
|experience_level|biggest_salary|employee_residence|
+----------------+--------------+------------------+
|          middle|      30400000|                CL|
|          senior|       7000000|                IN|
|            null|       6000000|                IN|
|            null|       4450000|                JP|
+----------------+--------------+------------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [94]:
df_final_filtered = df_final.filter(df_final["experience_level"].isNotNull())

df_final = df.join(df_final_filtered,
                     (df["salary"]== df_final_filtered["biggest_salary"]) &
                     (df["employee_residence"] == df_final_filtered["employee_residence"]))

print df_final

In [95]:
display(df_final.toPandas())

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,experience_level,biggest_salary,employee_residence
0,177,2021,MI,FT,Data Scientist,30400000,CLP,40038,CL,100,CL,L,middle,30400000,CL
1,285,2021,SE,FT,Data Science Manager,7000000,INR,94665,IN,50,IN,L,senior,7000000,IN


last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [96]:
biggest_salary_variable = df_final.select(max('salary')).collect()[0][0]
print(biggest_salary_variable)

30400000


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system